In [2]:
import os
WORKING_DIRECTORY = os.getcwd()
print("Current working directory: {}".format(WORKING_DIRECTORY))

Current working directory: /home/gabriel/Personal/jupyter/kaggle/dogsandcats/notebooks


In [6]:
ORIGINAL_TRAIN_DIRECTORY = "../data/original_train/"
TRAIN_DIRECTORY = "../data/train/"
VALID_DIRECTORY = "../data/valid/"
TEST_DIRECTORY = "../data/test/"

CLASSES = ['cat', 'dog']

VALIDATION_SIZE = 0.2 # size of the validation we want to use
TEST_SIZE = 0.1

## Data size

In [63]:
import pandas as pd
from PIL import Image

files = glob.glob(ORIGINAL_TRAIN_DIRECTORY + '*')
df = pd.DataFrame({'fpath':files,'w':0,'h':0})
df['category'] = df.fpath.str.extract('../data/original_train/([a-zA-Z]*).', expand=False) # extract class
for idx in df.index:
    im = Image.open(df.ix[idx].fpath)
    df.ix[idx,['w','h']] = im.size

In [64]:
df.head()

,fpath,h,w,category
0,../data/original_train/dog.798.jpg,293,249,dog
1,../data/original_train/dog.10748.jpg,301,349,dog
2,../data/original_train/dog.8365.jpg,375,499,dog
3,../data/original_train/cat.3227.jpg,443,291,cat
4,../data/original_train/dog.9597.jpg,221,294,dog


In [65]:
df.describe()

,h,w
count,25000.000000,25000.00000
mean,360.478080,404.09904
std,97.019959,109.03793
min,32.000000,42.00000
25%,301.000000,323.00000
50%,374.000000,447.00000
75%,421.000000,499.00000
max,768.000000,1050.00000


There are 25000 images in the dataset. We can see that the mean size of the images is (360.478080,404.09904). 

## Data preparation

The dataset can be downloaded from https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/data.

Number of training examples:

In [40]:
import os
TOTAL_NUMBER_FILES = sum([len(files) for r, d, files in os.walk(ORIGINAL_TRAIN_DIRECTORY)])
print("Total number of files in train folder:", TOTAL_NUMBER_FILES)

Total number of files in train folder: 25000


### Folder structure

The train directory consist of labelled data with the following convention for each image: 

data/train/CLASS.id.jpg

We are going to use keras.preprocessing.image so we want the folder structure to be:

data/train/CLASS/image-name.jpg

In [10]:
import glob
import os
import shutil
import numpy as np

shutil.rmtree(os.path.join(TEST_DIRECTORY, "dog"), ignore_errors=True)
shutil.rmtree(os.path.join(TEST_DIRECTORY, "cat"), ignore_errors=True)

shutil.rmtree(os.path.join(VALID_DIRECTORY, "dog"), ignore_errors=True)
shutil.rmtree(os.path.join(VALID_DIRECTORY, "cat"), ignore_errors=True)

shutil.rmtree(os.path.join(TRAIN_DIRECTORY, "dog"), ignore_errors=True)
shutil.rmtree(os.path.join(TRAIN_DIRECTORY, "cat"), ignore_errors=True)

os.mkdir(os.path.join(TEST_DIRECTORY, "dog"))
os.mkdir(os.path.join(TEST_DIRECTORY, "cat"))

os.mkdir(os.path.join(VALID_DIRECTORY, "dog"))
os.mkdir(os.path.join(VALID_DIRECTORY, "cat"))

os.mkdir(os.path.join(TRAIN_DIRECTORY, "dog"))
os.mkdir(os.path.join(TRAIN_DIRECTORY, "cat"))



In [11]:
#########################
# DOGS
##########
#random list of dog files
dog_pattern = ORIGINAL_TRAIN_DIRECTORY + "dog.*"
dog_files = np.random.permutation(glob.glob(dog_pattern))

# randomly split the files in train folder and move them to validation
number_validation_dog_files = int(len(dog_files) * VALIDATION_SIZE)
number_test_dog_files = int(len(dog_files) * TEST_SIZE)

for index, dog_file in enumerate(dog_files):
    file_name = os.path.split(dog_file)[1]
    if index < number_validation_dog_files:#validation files
        new_path = os.path.join(VALID_DIRECTORY, "dog",  file_name)
    elif index >= number_validation_dog_files and index < (number_validation_dog_files + number_test_dog_files):
        new_path = os.path.join(TEST_DIRECTORY, "dog",  file_name)
    else:
        new_path = os.path.join(TRAIN_DIRECTORY, "dog",  file_name)
    shutil.copy(dog_file, new_path)

#########################
# CATS
##########
#random list of dog files
cat_pattern = ORIGINAL_TRAIN_DIRECTORY + "cat.*"
cat_files = np.random.permutation(glob.glob(cat_pattern))

# randomly split the files in train folder and move them to validation
number_validation_cat_files = int(len(cat_files) * VALIDATION_SIZE)
number_test_cat_files = int(len(cat_files) * TEST_SIZE)

for index, cat_file in enumerate(cat_files):
    file_name = os.path.split(cat_file)[1]
    if index < number_validation_cat_files:
        new_path = os.path.join(VALID_DIRECTORY, "cat",  file_name)
    elif index >= number_validation_cat_files and index < (number_validation_cat_files+number_test_cat_files):
        new_path = os.path.join(TEST_DIRECTORY, "cat",  file_name)
    else:
        new_path = os.path.join(TRAIN_DIRECTORY, "cat",  file_name)
    shutil.copy(cat_file, new_path)    